In [1]:
import requests 
from bs4 import BeautifulSoup as bs
import os as os
import re
import json
import xml.etree.ElementTree as ET

def get_id_from_link(link):
# requesting data from website index
    r=requests.get(link)
    index = bs(r.content)

# grab disease id
    body=index.p.text
    body=body[7:]
    temp=""
    array=[]
    for char in body:
        if(char=="{"):
            temp=""
        elif(char=="}"):
            array.append(temp)
        else:
            temp=temp+char
            
# pass disease id       
    return array
    
def get_first_subElement(link):
    
# link = grabbed from network traffic (javascript , ajax)
    getbyID=requests.get(link)
    soup=bs(getbyID.content)

# grab disease , omim , protein,expasy, summaries(every variable in the webpage which doesn't required any array to store)
# try and catch were used in case there are empty data and broken webpage layout , example shown in the screenshot folder



    disease=soup.find('tr').find('td',"tr0").text
    try:
        synonyms=soup.find("tr","tr1").find("td","tr1").text
    except:
        synonyms=""

    array=[]
    try:
        omim_element = soup.find('a', href=re.compile("omim"))
        
        omim_url = omim_element['href']
        omim_id= omim_element.text
        
    except:
        omim_id=""
        omim_url=""
        
    try:   
        orphanet_element = soup.find('a', href=re.compile("orpha"))
        
        orphanet_url = orphanet_element['href']
        orphanet_number= orphanet_element.text

    except:
        orphanet_number=""
        orphanet_url=""
    try:
        protein_element = soup.find('a', href=re.compile("uniprot"))
        
        protein_url =  protein_element['href']
        protein= protein_element.text
       
    except:
        protein=""
        protein_url=""
    try:
        expasy_element = soup.find('a', href=re.compile("expasy"))
        expasy= expasy_element.text
        expasy_url =  expasy_element['href']
        
    except:
        expasy=""
        expasy_url=""
        

    flag=0
    for td in soup.find_all("td","lable"):

        if(flag!=0):
            if(flag==1):
                gene=td.text
            if(flag==2):
                icd=td.text
            flag=0
            
        if(td.text=="Gene locus"):
            flag=1
        if(td.text=="ICD"):
            flag=2
    
    # summary
    flag=0
    array=[]
    
    
    for td in soup.find_all("td","lable"):
        if(flag!=0):
            if(flag==1):
                summaries=td
            flag=0
            
        if(td.text=="Summary"):
            flag=1

    try:
        summaries = summaries.get_text(separator=";").strip()
    except:
        summaries=""
    
    disease=disease.replace("\r\n","")
    synonyms=synonyms.replace("\r\n","")
    omim_id=omim_id.replace("\r\n","")
    orphanet_number=orphanet_number.replace("\r\n","")
    protein=protein.replace("\r\n","")
    expasy=expasy.replace("\r\n","")
    gene=gene.replace("\r\n","")
    icd=icd.replace("\r\n","")
    summaries=summaries.replace("\r\n","")
    
    
    
    return(disease,synonyms,omim_id,omim_url,orphanet_number,orphanet_url,protein,protein_url,expasy,expasy_url,gene,icd,summaries)

def get_symptoms(link):
    
    # link = grabbed from network traffic (javascript , ajax)
    
    get_symptoms=requests.get(link)
    index=bs(get_symptoms.content)
    
    #symtomps tabs consist of json which is passed using ajax
    #json library is loaded to grab the json content
    data=json.loads(get_symptoms.content)

    symptoms=[]
    for row in data['rows']:
        symptom_id=row['id']
        symptom_name=row['data'][0]
        category=row['data'][1]
        symptoms.append([symptom_id,symptom_name,category])
        
    return symptoms


def get_metabolites(normal_link,lab_link):
    
    # link = grabbed from network traffic (javascript , ajax)
    get_normal_data=requests.get(normal_link)
    get_lab_data=requests.get(lab_link)
    
    #normal tabs consist of json which is passed using ajax
    #json library is loaded to grab the json content
    
    normal_data=json.loads(get_normal_data.content)
    
    
    
    #lab tabs consist of byte data which is in an almost similar format with json 
    #several tweaks are done to the data in order to match jsaon data format
    
    # convert lab data to json
    
    lab_data = str(get_lab_data.content)
    lab_data=lab_data.replace('rows', '"rows"')
    lab_data=lab_data.replace('{id', '{"id"')
    lab_data=lab_data.replace('data', '"data"')
    lab_data=lab_data[2:] 
    lab_data=lab_data[:-1]
    lab_data=lab_data.encode('utf_8')
    
    #load json data     
    lab_data=json.loads(lab_data)

    metabolites=[]
    
    metabolite_id=""
    name=""
    specimen=""
    _min=""
    _max=""
    unit=""
    age=""
    method=""
    comment=""
    value=""
    
    for row in normal_data['rows']:
        metabolite_id=row['id']
        name=row['data'][0]
        specimen=row['data'][4]
        _min=row['data'][1]
        _max=row['data'][2]
        unit=row['data'][3]
        age=row['data'][5]
        method=row['data'][6]
        comment=row['data'][7]
        
        for lab_row in lab_data['rows']:

            if(lab_row['data'][0]== row['data'][0] and lab_row['data'][1]==row['data'][4]):
                value=lab_row['data'][2]
        

        metabolites.append([metabolite_id,name,specimen,_min,_max,unit,age,value,method,comment])
    return metabolites

def get_literatures(link):
    # link = grabbed from network traffic (javascript , ajax)
    get_literature=requests.get(link)
    #load json data   
    literature_data=json.loads(get_literature.content)

    literatures=[]
    for row in literature_data['rows']:
        lit_id=row["id"]
    #the reason of this try and catch are shown on screenshot folder
        try:
            string=row['data'][0].split(">")[1]
            title=string.replace("</a","")
        except:
            title=row['data'][0]
        author=row['data'][1]
        journal=row['data'][2]
        year=row['data'][3]
        book=row['data'][4]
        volume=row['data'][5]
        number=row['data'][6]
        pages=row['data'][7]
        co_aut=row['data'][8]
        
        
        literatures.append([lit_id,title,author,journal,year,book,volume,number,pages,co_aut])

    return literatures

def generatexml(i,elements,symptoms_data,metabolites_data,literatures_data):
    
# level 1
    root=ET.Element("Disease")
# level 2
    name=ET.Element("Name")
    synonym=ET.Element("Synonym")
    omim=ET.Element("OMIM")
    orphanet=ET.Element("Orphanet")
    protein=ET.Element("Protein")
    expasy=ET.Element("ExPASy")
    gene=ET.Element("Gene")
    icd=ET.Element("ICD")
    summary=ET.Element("Summary")
    symptoms=ET.Element("Symptoms")
    metabolites=ET.Element("Metabolites")
    literatures=ET.Element("Literatures")
    
    root.append(name)
    root.append(synonym)
    root.append(omim)
    root.append(orphanet)
    root.append(protein)
    root.append(expasy)
    root.append(gene)
    root.append(icd)
    root.append(summary)
    root.append(symptoms)
    root.append(metabolites)
    root.append(literatures)
    
# level 3 (wihout loop)
    omim_id=ET.SubElement(omim,"Number")
    omim_url=ET.SubElement(omim,"URL")
    
    orphanet_number=ET.SubElement(orphanet,"Number")
    orphanet_url=ET.SubElement(orphanet,"URL")
    
    protein_number=ET.SubElement(protein,"Number")
    protein_url=ET.SubElement(protein,"URL")
    
    expasy_number=ET.SubElement(expasy,"Number")
    expasy_url=ET.SubElement(expasy,"URL")
    
    name.text=elements[0]
    synonym.text=elements[1]
    omim_id.text=elements[2]
    omim_url.text=elements[3]
    orphanet_number.text=elements[4]
    orphanet_url.text=elements[5]
    protein_number.text=elements[6]
    protein_url.text=elements[7]
    expasy_number.text=elements[8]
    expasy_url.text=elements[9]
    gene.text=elements[10]
    icd.text=elements[11]
    summary.text=elements[12]
    
#level 2 and 3 (with loop) = symtomps , metabolites , literatures


    for symptom_row in symptoms_data:
        symptom=ET.SubElement(symptoms,"symtomp")

        symptom_id=ET.SubElement(symptom,"id")
        symptom_name=ET.SubElement(symptom,"symptom")
        category=ET.SubElement(symptom,"category")

        symptom_id.text=str(symptom_row[0])
        symptom_name.text=str(symptom_row[1])
        category.text=str(symptom_row[2])
    
    
    for metabolite_row in metabolites_data:
        metabolite=ET.SubElement(metabolites,"metabolite")

        metabolite_id=ET.SubElement(metabolite,"id")
        name=ET.SubElement(metabolite,"name")
        specimen=ET.SubElement(metabolite,"specimen")
        value=ET.SubElement(metabolite,"value")
        _min=ET.SubElement(metabolite,"min")
        _max=ET.SubElement(metabolite,"max")
        unit=ET.SubElement(metabolite,"unit")
        age=ET.SubElement(metabolite,"age")
        method=ET.SubElement(metabolite,"method")
        comment=ET.SubElement(metabolite,"comment")
      
        metabolite_id.text=str(metabolite_row[0])
        name.text=str(metabolite_row[1])
        specimen.text=str(metabolite_row[2])
        value.text=str(metabolite_row[7])
        _min.text=str(metabolite_row[3])
        _max.text=str(metabolite_row[4])
        unit.text=str(metabolite_row[5])
        age.text=str(metabolite_row[6])
        method.text=str(metabolite_row[8])
        comment.text=str(metabolite_row[9])
        
    for literature_row in literatures_data:
        literature=ET.SubElement(literatures,"literature")

        lit_id=ET.SubElement(literature,"id")
        title=ET.SubElement(literature,"title")
        author=ET.SubElement(literature,"author")
        journal=ET.SubElement(literature,"journal")
        year=ET.SubElement(literature,"year")
        book=ET.SubElement(literature,"book")
        volume=ET.SubElement(literature,"volume")
        number=ET.SubElement(literature,"number")
        pages=ET.SubElement(literature,"pages")
        co_aut=ET.SubElement(literature,"co_aut")
      
        lit_id.text=str(literature_row[0])
        title.text=str(literature_row[1])
        author.text=str(literature_row[2])
        journal.text=str(literature_row[3])
        year.text=str(literature_row[4])
        book.text=str(literature_row[5])
        volume.text=str(literature_row[6])
        number.text=str(literature_row[7])
        pages.text=str(literature_row[8])
        co_aut.text=str(literature_row[9])


    path_parent = os.path.dirname(os.getcwd())
    path=path_parent+"/Output/XML-TemplateC/Disease-{}.xml".format(i)
    tree=ET.ElementTree(root)


    with open(path,"wb") as files:
            tree.write(path)
    
    
# get all disease id from index link
array=get_id_from_link('https://www.metagene.de/v2020/diseases.prg?act=getData&dhxr1605161946003=1')
j=0
for i in array:
# string manipulation to get id
# i = extracted id , j = xml output name
    i=(i.split(",")[0]).split(":")[1]
    print(j,i)
    try:
# get data from website
        first_subElement=get_first_subElement('https://www.metagene.de/v2020/diseases.prg?act=show&id={}&dhxr1605187011053=1'.format(i))
        symptoms=get_symptoms('https://www.metagene.de/v2020/symptoms.prg?act=getGridData&id_d={}&dhxr1605263478974=1'.format(i))
        metabolites=get_metabolites('https://www.metagene.de/appl/normal.prg?act=getData&id_d={}&dhxr1605267281056=1'.format(i),'https://www.metagene.de/v2020/lab.prg?act=getGridData2&id_d={}&dhxr1605268362441=1'.format(i))
        literatures=get_literatures('https://www.metagene.de/appl/authors.prg?act=getGridData&withLink=1&id_d={}&dhxr1605281356665=1'.format(i))
# generate xml
        generatexml(j,first_subElement,symptoms,metabolites,literatures)
    except:
        j=j-1
    j=j+1
    
        

    






0 458
1 1098
2 437
3 312
4 281
5 550
6 256
7 34
8 401
9 457
10 479
11 364
12 95
13 4
14 484
15 393
16 426
17 307
18 651
19 27
20 23
21 24
22 356
23 338
24 816
25 535
26 581
27 582
28 815
29 521
30 377
31 919
32 306
33 930
34 379
35 387
36 462
37 1019
38 1020
39 1233
40 411
41 47
42 414
43 406
44 402
45 567
46 1055
47 362
48 321
49 235
50 45
51 835
52 199
53 882
54 1181
55 660
56 311
57 53
58 892
59 380
60 360
61 21
62 507
63 370
64 357
65 516
66 769
67 747
68 823
69 822
70 968
71 1197
72 618
73 324
74 526
75 326
76 527
77 60
78 25
79 429
80 1110
81 354
82 804
83 906
84 1074
85 65
86 408
87 1024
88 732
89 1198
90 841
91 1109
92 1218
93 1217
94 288
95 796
96 797
97 795
98 798
99 799
100 800
101 802
102 69
103 1140
104 289
105 291
106 888
107 5
108 14
109 26
110 663
111 662
112 661
113 20
114 1025
115 1167
116 73
117 300
118 959
119 839
120 624
121 962
122 446
123 1017
124 941
125 195
126 77
127 1052
128 1011
129 80
130 1153
131 282
132 263
133 1114
134 94
135 1115
136 90
137 342
138 82
1

1012 1187
1013 1036
1014 1188
1015 957
1016 1027
1017 1012
1018 758
1019 1180
1020 1062
1021 914
1022 7
1023 119
1024 118
1025 1118
1026 984
1027 37
1028 405
1029 636
1030 494
1031 1230
1032 767
1033 768
1034 761
1035 746
1036 1064
1037 978
1038 1085
1039 909
1040 477
1041 809
1042 323
1043 482
1044 591
1045 70
1046 390
1047 425
1048 439
1049 49
1050 66
1051 319
1052 318
1053 453
1053 982
1054 718
1055 979
1056 586
1057 392
1058 956
1059 964
1060 981
1061 963
1062 1173
1063 384
1064 178
1065 534
1066 376
1067 710
1068 736
1069 597
1070 59
1071 483
1072 1015
1073 753
1074 1186
1075 673
1076 588
1077 1149
1078 677
1079 348
1080 1051
1081 347
1082 594
1083 1044
1084 1082
1085 897
1086 905
1087 895
1088 901
1089 903
1090 1178
1091 1043
1092 900
1093 902
1094 902
